<a href="https://colab.research.google.com/github/shaheerzubery/NLP-/blob/main/Langchain_QnA_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Our today's lecture is about langchain an we cover Question and answer module today

LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an API, but will also:

1.Be data-aware: connect a language model to other sources of data.

2.Be agentic: allow a language model to interact with its environment

The LangChain framework is designed with the above principles in mind.

This is the Python specific portion of the documentation. For a purely conceptual guide to LangChain

These are some of the Libraries which we need to install

In [1]:
!pip install python-magic
!pip install langchain
!pip install unstructured
!pip install openai
!pip install chromadb
!pip install langchain
!pip install pdfminer.six
!pip install unstructured[local-inference]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Libraries

In [24]:
pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.3 MB/s eta 0:00:00


Download some Dependencies

In [2]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langchain.document_loaders import OnlinePDFLoader

In [5]:
import pandas as pd
import nltk 
import magic
import os

#Large Language Model (LLM)

A large language model (LLM) is a language model consisting of a neural network with many parameters (typically billions of weights or more), trained on large quantities of unlabelled text using self-supervised learning. LLMs emerged around 2018 and perform well at a wide variety of tasks. This has shifted the focus of natural language processing research away from the previous paradigm of training specialized supervised models for specific tasks

So, LLM is a large language models like GPT 3.5 turbo, Devinci 003 GPT 4

We just connected our open AI with it

In [31]:
os.environ['OPENAI_API_KEY'] = Open_AI


This pdf we are using , you can use any document any json file 

In [32]:
loader = UnstructuredFileLoader("/content/Project Mgt.pdf")

In [33]:
documents = loader.load()

In [15]:
print(documents)

[Document(page_content="3601FM.fm  Page i  Monday, July 18, 2005  1:20 PM\n\n®\n\n:\n\nPMP\n\nProject Management\n\nProfessional\n\nStudy Guide\n\n3rd Edition\n\nKim Heldman, PMP\n\nWiley Publishing, Inc.\n\n3601FM.fm  Page ii  Monday, July 18, 2005  1:20 PM\n\nCopyright © 2005 by Wiley Publishing, Inc., Hoboken, New Jersey\n\nPublished simultaneously in Canada.\n\nISBN\n\n\n\n13: 978\n\n\n\n0\n\n\n\n7821\n\n\n\n3601\n\n\n\nX\n\nISBN\n\n\n\n10: 0\n\n\n\n7821\n\n\n\n3601\n\n\n\nX\n\nNo part of this publication may be reproduced, stored in a retrieval system or transmitted in any form or by any means, electronic, mechanical, photocopying, recording, scanning or otherwise, except as permitted under Sec- tions 107 or 108 of the 1976 United States Copyright Act, without either the prior written permission of the Pub- lisher, or authorization through payment of the appropriate per-copy fee to the Copyright Clearance Center, 222 Rosewood Drive, Danvers, MA 01923, (978) 750-8400, fax (978) 646

After loading the pdf we are chunking the document into small parts and then using OpenAi to create the embedding of it, the old method of creating emdedding was word embedding

In [34]:
text_splitter = CharacterTextSplitter(chunk_size=3523, chunk_overlap=100)


In [35]:
texts = text_splitter.split_documents(documents)


In [36]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])


After Creating Embeddings we are storing it into vector Store,
we are using chroma there are multiple that you can use like deep lake, pinecone or elastic search

In [37]:
docsearch = Chroma.from_documents(documents=texts, embedding=embeddings)

In [38]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(temperature = 0), chain_type="stuff", vectorstore=docsearch, return_source_documents=True,verbose=True, 
    )


/usr/local/lib/python3.9/dist-packages/langchain/chains/retrieval_qa/base.py:186: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [39]:

query = """Project selection methods 
"""


result = qa({"query": query})



> Entering new VectorDBQA chain...

> Finished chain.


In [40]:
result['result']


' Project selection methods are concerned with the advantages or merits of the product of the project. They measure the value of what the product or service of the project will produce and how it will benefit the organization. Selection methods involve the types of concerns executive managers are typically thinking about, such as market share, financial benefits, return on investment, customer retention and loyalty, and public perceptions. They are used to evaluate and choose between alternative ways of performing the project.'

Source document gives the multples result which are closer to the required prompt

In [41]:
result['source_documents']

[Document(page_content='The project selection methods we’ll talk about next are ones you should know and understand for the exam. However, keep in mind they are only one aspect of project selec- tion in the real world. The individual opinion, and power, of selection committee members also plays a part in the projects the organization chooses to perform. Don’t underestimate the importance of the authority, political standing, and individual aspirations of selection committee members. Those committee members who happen to carry a lot of weight in company circles, so to speak, are likely to get their projects approved just on the fact that they are who they are. This is sometimes how project selection works in my organization. How about yours?\n\n3601.book  Page 59  Wednesday, July 13, 2005  1:39 PM\n\nUsing Tools and Techniques for Charter Development\n\n59\n\nUsing Project Selection Methods\n\nProject selection methods are concerned with the advantages or merits of the product of the pr

### Key

In [42]:
Open_AI = ''